In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib as mpl
from scipy.stats import norm

# 中文乱码的处理
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']# 设置微软雅黑字体
plt.rcParams['axes.unicode_minus'] = False # 避免坐标轴不能正常的显示负号

In [48]:
import pandas as pd
from sklearn.model_selection import train_test_split
# from xgboost import XGBRegressor

# Road the data
X_full = pd.read_csv('./data/train.csv')
X_test_full = pd.read_csv('./data/test.csv')
sub = pd.read_csv('./data/sub.csv')
print(X_full.shape, X_test_full.shape)
#$$


(59288, 26) (14823, 25)


In [49]:
# X_full.head();
X_full.columns

Index(['数据ID', '容纳人数', '便利设施', '洗手间数量', '床的数量', '床的类型', '卧室数量', '取消条款', '所在城市',
       '清洁费', '首次评论日期', '房主是否有个人资料图片', '房主身份是否验证', '房主回复率', '何时成为房主',
       '是否支持随即预订', '最近评论日期', '维度', '经度', '民宿周边', '评论个数', '房产类型', '民宿评分', '房型',
       '邮编', '价格'],
      dtype='object')

In [50]:
import re
words = [re.findall("\w+",  str) for str in X_full.便利设施]

In [51]:
#

add new col, count for bianlisheshi


In [52]:
X_full['new'] = 1
for i in range(0, len(X_full)):
    X_full.loc[i,'new'] = len((X_full.便利设施[i]).split())

In [53]:
X_full['new_len'] = 1
for i in range(0, len(X_full)):
    X_full.loc[i,'new_len'] = len((X_full.便利设施[i]))

In [54]:
#

check na

In [55]:
# na_index = [X_full[col].isnull().values.sum() for col in X_full.columns]
# na_key = X_full.columns
# na_frame = pd.DataFrame(na_index, na_key)

Cat and Num split


In [56]:
X_full = X_full.drop('数据ID', axis = 1)

In [57]:
categorical_cols = [cname for cname in X_full.columns if 
X_full[cname].dtype == 'object']

# Select numerical columns
numerical_cols = [cname for cname in X_full.columns if
                  X_full[cname].dtype in ['float64', 'int64']]

In [58]:
categorical_cols

['便利设施',
 '首次评论日期',
 '房主是否有个人资料图片',
 '房主身份是否验证',
 '房主回复率',
 '何时成为房主',
 '最近评论日期',
 '民宿周边',
 '邮编']

In [59]:
X_full.首次评论日期 = [str(i) for i in X_full.首次评论日期]
X_full.房主回复率  = [str(i) for i in X_full.房主回复率]

In [60]:
X_full.首次评论日期 = [re.findall("^\d{4}\-\d{2}", str) for str in X_full.首次评论日期]


In [61]:
X_full.房主回复率 = [re.findall("^\d+", str) for str in X_full.房主回复率]

In [81]:
int(X_full.房主回复率[2][0])

100

In [80]:
#


In [71]:
X_full.房主回复率_new  = [int(i[0]) for i in X_full.房主回复率]

IndexError: list index out of range

#--------------------------------------------
drop

In [ ]:
X_full.drop['便利设施', '']

In [84]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [76]:
y = X_full['价格']
X_full.pop('价格')

0        64.918531
1        54.918531
2        73.219281
3        64.093909
4        68.454901
           ...    
59283    57.813597
59284    62.288187
59285    73.128830
59286    62.854022
59287    64.093909
Name: 价格, Length: 59288, dtype: float64

In [85]:
numerical_transformer = SimpleImputer(strategy='most_frequent')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [86]:
X_train_new = preprocessor.fit_transform(X_full)

In [87]:
X_train_new

<59288x62551 sparse matrix of type '<class 'numpy.float64'>'
	with 1370374 stored elements in Compressed Sparse Row format>

In [88]:
# X_full.columns

cat one hot

In [65]:
# from sklearn.model_selection import train_test_split

In [ ]:
# X_train_full, X_valid_full, y_train, y_valid = 
# train_test_split(X_full, y, train_size=0.8, test_size=0.2, random_state=0)

In [112]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import randint as sp_randint


In [93]:
my_reg = RandomForestRegressor()
scores = -1 * cross_val_score(my_reg, 
    X_train_new, y,
    cv=5, n_jobs = -1,
    scoring='neg_root_mean_squared_error')

print("MAE scores:\n", scores)
print("Average MAE score (across experiments):")
print(scores.mean())

MAE scores:
 [5.73508443 5.72922823 5.63633206 5.55389031 5.81713822]
Average MAE score (across experiments):
5.694334648711674


In [114]:
hp = {"max_depth": [14, 11, 9],
              "min_samples_split": sp_randint(2, 11)
            #   "criterion": ["gini", "entropy"],
            #   "bootstrap": [True, False]
              }

# run randomized search
n_iter_search = 10
random_search = RandomizedSearchCV(my_reg, param_distributions=hp,
                                   n_iter=n_iter_search, cv=5)



In [117]:
 
#% Initial Models
# from sklearn.ensemble import RandomForestRegressor
# import lightgbm as lgb

RFReg = RandomForestRegressor(n_estimators = 10, criterion = "mse", max_depth = 10, n_jobs = -1,random_state = 0).fit(X_train_new, y)



RandomForestRegressor(max_depth=10, n_estimators=10, n_jobs=-1, random_state=0)

test data

In [94]:
X_test_full['new'] = 1
for i in range(0, len(X_test_full)):
    X_test_full.loc[i,'new'] = len((X_test_full.便利设施[i]).split())

In [95]:
X_test_full['new_len'] = 1
for i in range(0, len(X_test_full)):
    X_test_full.loc[i,'new_len'] = len((X_test_full.便利设施[i]))

In [96]:
X_test_full = X_test_full.drop('数据ID', axis = 1)

In [97]:
categorical_cols = [cname for cname in X_test_full.columns if 
X_test_full[cname].dtype == 'object']

# Select numerical columns
numerical_cols = [cname for cname in X_test_full.columns if
                  X_test_full[cname].dtype in ['float64', 'int64']]

In [101]:
# y = X_test_full['价格']
# X_test_full.pop('价格')

In [121]:
X_test_new = preprocessor.transform(X_test_full)

In [122]:
predicted_ = RFReg.predict(X_test_new)

ValueError: X has 21067 features, but DecisionTreeRegressor is expecting 62551 features as input.

In [123]:
X_test_new

<14823x21067 sparse matrix of type '<class 'numpy.float64'>'
	with 342605 stored elements in Compressed Sparse Row format>